In [102]:
#import libraries
import pandas as pd
from sklearn.metrics import *
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import pickle
from sklearn.base import clone
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import (
    cross_val_score, StratifiedKFold, 
    train_test_split, HalvingGridSearchCV
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [104]:
june_2021 = pd.read_csv('june_2021.csv')
july_2021 = pd.read_csv('july_2021.csv')

june_2022 = pd.read_csv('june_2022.csv')
july_2022 = pd.read_csv('july_2022.csv')     

In [105]:
#concatenate all dataframes
final_df = pd.concat([june_2021, july_2021, june_2022, july_2022], axis=0)


In [106]:
print("Data size: ", flights_df.shape)

Data size:  (546124, 61)


In [107]:
flights_df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS
0,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,4702,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,935,931.0,-4.0,0.0,0.0,-1.0,0900-0959,13.0,944.0,1110.0,8.0,1143,1118.0,-25.0,0.0,0.0,-2.0,1100-1159,0.0,0.0,128.0,107.0,86.0,1.0,509.0,3,0.0,0.0,0.0,0.0,0.0,0.0
1,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,4788,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,11193,1119302,33105,CVG,"Cincinnati, OH",KY,21,Kentucky,52,1238,1423.0,105.0,105.0,1.0,7.0,1200-1259,15.0,1438.0,1517.0,4.0,1347,1521.0,94.0,94.0,1.0,6.0,1300-1359,0.0,0.0,69.0,58.0,39.0,1.0,229.0,1,94.0,0.0,0.0,0.0,0.0,0.0
2,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,5238,11193,1119302,33105,CVG,"Cincinnati, OH",KY,21,Kentucky,52,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,1437,1605.0,88.0,88.0,1.0,5.0,1400-1459,11.0,1616.0,1803.0,4.0,1648,1807.0,79.0,79.0,1.0,5.0,1600-1659,0.0,0.0,131.0,122.0,107.0,1.0,773.0,4,0.0,0.0,0.0,0.0,79.0,0.0
3,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,5238,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,11193,1119302,33105,CVG,"Cincinnati, OH",KY,21,Kentucky,52,1740,1900.0,80.0,80.0,1.0,5.0,1700-1759,14.0,1914.0,2051.0,6.0,1953,2057.0,64.0,64.0,1.0,4.0,1900-1959,0.0,0.0,133.0,117.0,97.0,1.0,773.0,4,2.0,0.0,0.0,0.0,62.0,0.0
4,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,5311,14321,1432105,34321,PWM,"Portland, ME",ME,23,Maine,12,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,700,659.0,-1.0,0.0,0.0,-1.0,0700-0759,25.0,724.0,812.0,9.0,826,821.0,-5.0,0.0,0.0,-1.0,0800-0859,0.0,0.0,86.0,82.0,48.0,1.0,273.0,2,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
#check number of missing values column wise
print(flights_df.isna().sum())

YEAR                    0
QUARTER                 0
MONTH                   0
DAY_OF_MONTH            0
DAY_OF_WEEK             0
                       ..
WEATHER_DELAY           0
NAS_DELAY               0
SECURITY_DELAY          0
LATE_AIRCRAFT_DELAY     0
DIV_AIRPORT_LANDINGS    0
Length: 61, dtype: int64


In [109]:
flights_df.fillna(0, inplace=True)

In [110]:
flights_df.isna().sum().sum()

0

In [111]:
final_df = flights_df.copy()

In [112]:
import calendar
import datetime

flight_date = pd.DatetimeIndex(final_df['FL_DATE'])

final_df['DAY_OF_MONTH'] = final_df['DAY_OF_MONTH']
final_df['MONTH'] = final_df['MONTH']
final_df['DAY_OF_WEEK'] = final_df['DAY_OF_WEEK']

# Add a new column to display the day of the week
final_df['DayName'] = final_df['DAY_OF_WEEK'].apply(lambda x: calendar.day_name[x - 1])

# Add a new column to display whether it's a weekday or a weekend
final_df['Weekday'] = final_df['DAY_OF_WEEK'].apply(lambda x: 1 if x in range(0, 5) else 0)

final_df.head(2)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,DayName,Weekday
0,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,4702,12478,1247805,31703,JFK,"New York, NY",NY,36,New York,22,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,935,931.0,-4.0,0.0,0.0,-1.0,0900-0959,13.0,944.0,1110.0,8.0,1143,1118.0,-25.0,0.0,0.0,-2.0,1100-1159,0.0,0.0,128.0,107.0,86.0,1.0,509.0,3,0.0,0.0,0.0,0.0,0.0,0.0,Tuesday,1
1,2021,2,6,1,2,6/1/2021 12:00:00 AM,9E,20363,9E,N131EV,4788,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,11193,1119302,33105,CVG,"Cincinnati, OH",KY,21,Kentucky,52,1238,1423.0,105.0,105.0,1.0,7.0,1200-1259,15.0,1438.0,1517.0,4.0,1347,1521.0,94.0,94.0,1.0,6.0,1300-1359,0.0,0.0,69.0,58.0,39.0,1.0,229.0,1,94.0,0.0,0.0,0.0,0.0,0.0,Tuesday,1


# Preprocessing

In [113]:
# Using quantiles for threshold
def label_carrier_train(carrier):
    if carrier in ['WN', 'AA', 'DL', 'OO', 'UA']:
        return 3
    if carrier in ['YX', '9E', 'MQ', 'B6', 'OH']:
        return 2
    if carrier in ['AS', 'NK', 'F9', 'G4', 'YV', 'QX', 'HA']:
        return 1
    return 0


# Feature Engineering

In [141]:
final_df['DEP_TIME'].max()

2400.0

In [114]:
from sklearn.preprocessing import LabelEncoder

In [115]:
X_test

,MONTH,DAY_OF_MONTH,OP_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,DISTANCE_GROUP,Weekday,carrier_popularity,is_weekend
388655,6,22,8,200,251,1029.0,1059.0,2,1,2,0
495252,6,28,3,182,119,1003.0,1302.0,5,1,2,1
477268,6,27,4,237,95,1559.0,1735.0,3,0,3,1
440031,6,25,3,118,264,853.0,1104.0,9,0,2,0
398561,6,23,0,172,22,701.0,817.0,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...
481061,6,27,8,251,265,1910.0,1953.0,1,0,2,1
17831,6,2,1,96,213,1323.0,1443.0,2,1,3,0
387292,6,22,4,287,103,553.0,728.0,3,1,3,0
338223,6,19,14,95,272,1108.0,1612.0,5,0,3,0


In [116]:
cols_to_consider = ['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME', 
       'ARR_TIME', 
#        'DISTANCE_GROUP', 
       'Weekday', 'DEP_DELAY']
final_df = final_df[cols_to_consider]

In [117]:
flights_df.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID',
       'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_DELAY_GROUP', 'DEP_TIME_BLK', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK',
       'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
       'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY',
       

In [118]:
final_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,Weekday,DEP_DELAY
0,6,1,2,9E,JFK,DTW,931.0,1118.0,1,-4.0
1,6,1,2,9E,DTW,CVG,1423.0,1521.0,1,105.0
2,6,1,2,9E,CVG,TPA,1605.0,1807.0,1,88.0
3,6,1,2,9E,TPA,CVG,1900.0,2057.0,1,80.0
4,6,1,2,9E,PWM,JFK,659.0,821.0,1,-1.0
...,...,...,...,...,...,...,...,...,...,...
546119,6,30,3,YX,DTW,CLE,840.0,920.0,1,-5.0
546120,6,30,3,YX,EWR,MSP,1101.0,1302.0,1,16.0
546121,6,30,3,YX,MSP,IAD,1332.0,1725.0,1,17.0
546122,6,30,3,YX,IND,EWR,716.0,1017.0,1,1.0


In [119]:
def process_features(final_df):
    final_df = final_df.copy()
    # Create is_weekend col
    final_df['is_weekend'] = final_df['DAY_OF_WEEK'].isin([1, 7]).astype(int)

    # Drop day and time columns
    final_df.drop(['DAY_OF_WEEK'], axis=1, inplace=True)

    #Bin and OHE Carriers
    final_df['carrier_popularity'] = final_df['OP_CARRIER'].apply(lambda x: label_carrier_train(x))

#     # Get avg delay
#     final_df = add_mean_column(final_df, 'origin_avg_time.csv', 'ORIGIN')
#     final_df = add_mean_column(final_df, 'dest_avg_time.csv', 'DEST')
#     final_df = add_mean_column(final_df, 'tailnumber_avg_time.csv', 'TAIL_NUM')
    
    le = LabelEncoder()
    le.fit(final_df['OP_CARRIER'])
    final_df['OP_CARRIER'] = le.transform(final_df['OP_CARRIER'])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("OP_CARRIER_mapping", mapping)
    
    le = LabelEncoder()
    le.fit(final_df['ORIGIN'])
    final_df['ORIGIN'] = le.transform(final_df['ORIGIN'])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("ORIGIN_mapping", mapping)
    
    le = LabelEncoder()
    le.fit(final_df['DEST'])
    final_df['DEST'] = le.transform(final_df['DEST'])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("DEST_mapping", mapping)
#     final_df = final_df[((final_df['ARR_TIME']<=1440) & (final_df['DEP_TIME']<=1440))]
    
    # Impute nulls with 0
    final_df= final_df.fillna(0)

    return final_df

final_df_2 = process_features(final_df.copy())
final_df_2.head(5)


OP_CARRIER_mapping {'9E': 0, 'AA': 1, 'AS': 2, 'B6': 3, 'DL': 4, 'F9': 5, 'G4': 6, 'HA': 7, 'MQ': 8, 'NK': 9, 'OH': 10, 'OO': 11, 'QX': 12, 'UA': 13, 'WN': 14, 'YV': 15, 'YX': 16}
ORIGIN_mapping {'ABE': 0, 'ABI': 1, 'ABQ': 2, 'ABR': 3, 'ABY': 4, 'ACK': 5, 'ACT': 6, 'ACV': 7, 'ACY': 8, 'ADK': 9, 'ADQ': 10, 'AEX': 11, 'AGS': 12, 'AKN': 13, 'ALB': 14, 'ALO': 15, 'ALS': 16, 'ALW': 17, 'AMA': 18, 'ANC': 19, 'APN': 20, 'ASE': 21, 'ATL': 22, 'ATW': 23, 'ATY': 24, 'AUS': 25, 'AVL': 26, 'AVP': 27, 'AZA': 28, 'AZO': 29, 'BDL': 30, 'BET': 31, 'BFF': 32, 'BFL': 33, 'BGM': 34, 'BGR': 35, 'BHM': 36, 'BIL': 37, 'BIS': 38, 'BJI': 39, 'BKG': 40, 'BLI': 41, 'BLV': 42, 'BMI': 43, 'BNA': 44, 'BOI': 45, 'BOS': 46, 'BPT': 47, 'BQK': 48, 'BQN': 49, 'BRD': 50, 'BRO': 51, 'BRW': 52, 'BTM': 53, 'BTR': 54, 'BTV': 55, 'BUF': 56, 'BUR': 57, 'BWI': 58, 'BZN': 59, 'CAE': 60, 'CAK': 61, 'CDB': 62, 'CDC': 63, 'CDV': 64, 'CGI': 65, 'CHA': 66, 'CHO': 67, 'CHS': 68, 'CID': 69, 'CIU': 70, 'CKB': 71, 'CLE': 72, 'CLL': 73, 

,MONTH,DAY_OF_MONTH,OP_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,Weekday,DEP_DELAY,is_weekend,carrier_popularity
0,6,1,0,182,103,931.0,1118.0,1,-4.0,0,2
1,6,1,0,103,86,1423.0,1521.0,1,105.0,0,2
2,6,1,0,86,343,1605.0,1807.0,1,88.0,0,2
3,6,1,0,343,86,1900.0,2057.0,1,80.0,0,2
4,6,1,0,283,182,659.0,821.0,1,-1.0,0,2


In [120]:
final_df_2.shape

(546124, 11)

In [121]:
final_df_2.columns

Index(['MONTH', 'DAY_OF_MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME',
       'ARR_TIME', 'Weekday', 'DEP_DELAY', 'is_weekend', 'carrier_popularity'],
      dtype='object')

# Train Test Split

In [122]:
final_df_2['DEP_DEL15'] = np.where(final_df_2['DEP_DELAY'] > 15, 1, 0)

pre_depart_features = ['MONTH', 'DAY_OF_MONTH', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME', 'ARR_TIME', 'Weekday', 'carrier_popularity', 'is_weekend']
X = final_df_2[pre_depart_features]
y = final_df_2['DEP_DEL15']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=123)

X.to_csv('X')
y.to_csv('y')
X_train.to_csv('X_train')
X_test.to_csv('X_test')
y_train.to_csv('y_train')
y_test.to_csv('y_test')


(436899, 10)
(436899,)


In [123]:
X_test.head()

,MONTH,DAY_OF_MONTH,OP_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,Weekday,carrier_popularity,is_weekend
388655,6,22,8,200,251,1029.0,1059.0,1,2,0
495252,6,28,3,182,119,1003.0,1302.0,1,2,1
477268,6,27,4,237,95,1559.0,1735.0,0,3,1
440031,6,25,3,118,264,853.0,1104.0,0,2,0
398561,6,23,0,172,22,701.0,817.0,1,2,0


# Modelling

In [124]:
# transform_features = FunctionTransformer(process_features)

In [125]:
models_dict = {
    "log_reg": LogisticRegression(class_weight='balanced', random_state=55),
    "svm": LinearSVC(class_weight='balanced', random_state=55),
    "dtc": DecisionTreeClassifier(class_weight='balanced', random_state=55),
    "rfc": RandomForestClassifier(class_weight='balanced', random_state=55),
}
     

In [126]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

def create_pipeline(classifier):
    pipe = make_pipeline(
        MinMaxScaler(), 
        classifier
    )
    return pipe


In [127]:
X_train

,MONTH,DAY_OF_MONTH,OP_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,Weekday,carrier_popularity,is_weekend
164920,6,10,5,298,251,1034.0,1237.0,1,1,0
204650,6,12,10,74,265,1824.0,1926.0,0,2,0
430435,6,24,13,193,312,814.0,951.0,1,3,0
494473,6,28,1,96,252,2139.0,148.0,1,3,1
106849,6,7,1,301,251,728.0,1023.0,1,3,1
...,...,...,...,...,...,...,...,...,...,...
262291,6,15,11,103,72,1220.0,1301.0,1,3,0
539409,6,30,11,235,237,1515.0,1610.0,1,3,0
535141,6,30,6,267,35,1737.0,2045.0,1,1,0
199481,6,12,2,362,64,1153.0,1246.0,0,1,0


In [128]:
%%time

trained_pipes = {}
for key, classifier in models_dict.items():
    print(f"Model: {classifier}\n")

    pipe = create_pipeline(classifier)
    # fit the pipeline on the transformed data
    pipe.fit(X_train.copy(), y_train)

    trained_pipes[key] = pipe

    # make predictions
    y_pred = pipe.predict(X_test.copy())

    # evaluate pipeline
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)
    cv_scores = cross_val_score(
        pipe, 
        X_train.copy(), 
        y_train, 
        scoring='f1', 
        cv=cv, 
        n_jobs=-1
    )
    print(f"Cross validated mean F1 Score = {round(np.mean(cv_scores), 3)}")

    print(f"Balanced Accuracy on Test set: {round(balanced_accuracy_score(y_test, y_pred), 3)}")
    print(f"F1 Score on Test set: {round(f1_score(y_test, y_pred), 3)}\n")


    print(f"\nClassification report: \n{classification_report(y_test, y_pred, labels=[1, 0])}")
    print("----------------------------------------------------------------")

Model: LogisticRegression(class_weight='balanced', random_state=123)

Cross validated mean F1 Score = 0.469
Balanced Accuracy on Test set: 0.656
F1 Score on Test set: 0.467


Classification report: 
              precision    recall  f1-score   support

           1       0.36      0.66      0.47     25151
           0       0.87      0.65      0.74     84074

    accuracy                           0.65    109225
   macro avg       0.61      0.66      0.60    109225
weighted avg       0.75      0.65      0.68    109225

----------------------------------------------------------------
Model: LinearSVC(class_weight='balanced', random_state=123)

Cross validated mean F1 Score = 0.469
Balanced Accuracy on Test set: 0.656
F1 Score on Test set: 0.468


Classification report: 
              precision    recall  f1-score   support

           1       0.36      0.66      0.47     25151
           0       0.87      0.65      0.74     84074

    accuracy                           0.65    109225
 

In [129]:
trained_pipes     

{'log_reg': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('logisticregression',
                  LogisticRegression(class_weight='balanced',
                                     random_state=123))]),
 'svm': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('linearsvc',
                  LinearSVC(class_weight='balanced', random_state=123))]),
 'dtc': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('decisiontreeclassifier',
                  DecisionTreeClassifier(class_weight='balanced',
                                         random_state=123))]),
 'rfc': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(class_weight='balanced',
                                         random_state=123))])}

In [130]:
# save all pipelines
joblib.dump(trained_pipes, 'trained_pipes.pkl')
# save best model a file
joblib.dump(trained_pipes['rfc'], 'rf_clf.pkl')

['rf_clf.pkl']

# Hyperparameter Tuning

The HalvingGridSearchCV is utilised to speed up the hyperparameter tuning selection.

In [131]:
%%time

param_grid = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__max_features': ['sqrt', 'log2'],
    'randomforestclassifier__max_depth': [3, 5, 8],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
}

pipeline_rfc = make_pipeline(
#         transform_features, 
        MinMaxScaler(), 
        RandomForestClassifier(class_weight='balanced', random_state=123),
)

folds = StratifiedKFold(
    n_splits=3, 
    shuffle=True, 
    random_state=123
)

grid = HalvingGridSearchCV(
    pipeline_rfc, 
    param_grid, 
    scoring="f1", 
    cv=folds, 
    random_state=123, 
    n_jobs=-1, 
    verbose=2
)

# fitting the model for grid search
grid.fit(X_train.copy(), y_train)
     

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 16181
max_resources_: 436899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 16181
Fitting 3 folds for each of 54 candidates, totalling 162 fits
----------
iter: 1
n_candidates: 18
n_resources: 48543
Fitting 3 folds for each of 18 candidates, totalling 54 fits
----------
iter: 2
n_candidates: 6
n_resources: 145629
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 3
n_candidates: 2
n_resources: 436887
Fitting 3 folds for each of 2 candidates, totalling 6 fits
CPU times: user 10.2 s, sys: 519 ms, total: 10.7 s
Wall time: 1min 12s


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=123, shuffle=True),
                    estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                              ('randomforestclassifier',
                                               RandomForestClassifier(class_weight='balanced',
                                                                      random_state=123))]),
                    n_jobs=-1,
                    param_grid={'randomforestclassifier__max_depth': [3, 5, 8],
                                'randomforestclassifier__max_features': ['sqrt',
                                                                         'log2'],
                                'randomforestclassifier__min_samples_split': [2,
                                                                              5,
                                                                              10],
                                'randomforestclassifier__n_estimators': [50,
                                                                         100,
                                                                         200]},
                    random_state=123, scoring='f1', verbose=2)

In [132]:
# Finding best hyperparameters
print(f"Best Hyperparameters: \n{grid.best_params_}\n")
print(f"Mean CV Score of the best estimator: {round(grid.best_score_, 3)}")

Best Hyperparameters: 
{'randomforestclassifier__max_depth': 8, 'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__n_estimators': 50}

Mean CV Score of the best estimator: 0.517


In [133]:
# Score on test dataset
grid_predictions = grid.predict(X_test.copy())
print(
    f"Grid Search F1 Score on Test data: {round(f1_score(y_test, grid_predictions), 3)}"
)

Grid Search F1 Score on Test data: 0.514


In [134]:
print(classification_report(y_test, grid_predictions, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.41      0.68      0.51     25151
           0       0.88      0.71      0.79     84074

    accuracy                           0.70    109225
   macro avg       0.65      0.70      0.65    109225
weighted avg       0.77      0.70      0.72    109225



In [135]:
# Save fine-tuned model
joblib.dump(grid, 'rfc_hpt.pkl')     

['rfc_hpt.pkl']

[CV] END randomforestclassifier__max_depth=3, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=50; total time=   0.2s
[CV] END randomforestclassifier__max_depth=3, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_split=10, randomforestclassifier__n_estimators=100; total time=   0.3s
[CV] END randomforestclassifier__max_depth=3, randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=200; total time=   0.6s
[CV] END randomforestclassifier__max_depth=3, randomforestclassifier__max_features=log2, randomforestclassifier__min_samples_split=10, randomforestclassifier__n_estimators=200; total time=   0.6s
[CV] END randomforestclassifier__max_depth=5, randomforestclassifier__max_features=sqrt, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=50; total time=   0.2s
[CV] END randomforestclas